In [1]:
import torch

from transformers import AutoTokenizer, GPTNeoXForCausalLM as HF_GPTNeoXForCausalLM
# from optimum.bettertransformer import BetterTransformer

%load_ext autoreload
%autoreload 2
from peft_pretraining.modeling_pythia import GPTNeoXForCausalLM

/fsx/vlialin/relora/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/fsx/vlialin/relora/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [2]:
model_name = "EleutherAI/pythia-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids = tokenizer("Hello, my dog is cute", return_tensors="pt").input_ids

In [3]:
flashattention_model = GPTNeoXForCausalLM.from_pretrained(model_name)

In [4]:
flashattention_out = flashattention_model(input_ids)

In [5]:
orig_model = HF_GPTNeoXForCausalLM.from_pretrained(model_name)
# orig_model = BetterTransformer.transform(orig_model)

In [6]:
orig_out = orig_model(input_ids)

In [7]:
orig_out.logits

tensor([[[  3.6091,  -9.3721,   8.7746,  ...,  -9.2669,  -9.2728,  -9.3494],
         [  1.8715, -10.9088,   5.3397,  ..., -10.6625, -10.8846, -10.8984],
         [  1.1809, -10.6320,   6.5040,  ..., -10.4274, -10.7300, -10.4196],
         [  5.8298,  -8.4654,  12.3902,  ...,  -8.4833,  -8.6062,  -8.5475],
         [  3.7810,  -9.6519,   6.0415,  ...,  -9.6365,  -9.8250,  -9.5622],
         [  8.1836,  -8.4023,  14.3205,  ...,  -8.5621,  -8.4343,  -8.4242]]],
       grad_fn=<UnsafeViewBackward0>)

In [8]:
flashattention_out.logits

tensor([[[  3.6091,  -9.3721,   8.7746,  ...,  -9.2669,  -9.2728,  -9.3494],
         [  1.8715, -10.9088,   5.3397,  ..., -10.6625, -10.8846, -10.8984],
         [  1.1809, -10.6320,   6.5040,  ..., -10.4274, -10.7300, -10.4196],
         [  5.8298,  -8.4654,  12.3902,  ...,  -8.4833,  -8.6062,  -8.5475],
         [  3.7810,  -9.6519,   6.0415,  ...,  -9.6365,  -9.8250,  -9.5622],
         [  8.1836,  -8.4023,  14.3205,  ...,  -8.5621,  -8.4343,  -8.4242]]],
       grad_fn=<UnsafeViewBackward0>)

In [10]:
torch.allclose(orig_out.logits, flashattention_out.logits, atol=1e-5)

True

In [22]:
out_gen = orig_model.generate(input_ids)
tokenizer.decode(out_gen[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/mnt/shared_home/vlialin/miniconda3/envs/peft_pretraining_shala/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Hello, my dog is cute, but he\'s not a dog. He\'s a cat."\n'

In [18]:
out_gen = flashattention_model.generate(input_ids)
tokenizer.decode(out_gen[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Hello, my dog is cute, I am. I am. I am. I am. I'